In [2]:
!pip install swyft
!pip install camb

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
from tqdm import tqdm
import swyft
import camb
import pandas as pd
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

In [4]:
DEVICE

'cpu'

In [5]:
class Network3(swyft.SwyftModule):
    def __init__(self):
        super().__init__()
        self.learning_rate = 1e-6
        self.net = torch.nn.Sequential(
            torch.nn.Linear(128, 256),
            #torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 512),
            #torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            #torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            #torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            #torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            #torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 16),
            #torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 8),
            #torch.nn.Dropout(0.05),
            torch.nn.ReLU())
        self.logratios = swyft.LogRatioEstimator_1dim(num_features = 8, num_params = 5, varnames = 'z')

    def forward(self, A, B):
      s = self.net(A['x'])
      return self.logratios(s, B['z']), s

In [6]:
cls = np.load('cls.npy')
pars = np.load('params.npy')

In [7]:
cls_short = [item[:128] for item in cls]

In [8]:
def generate_camb_cl(H0=67.5, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0,
                     tau=0.06, logA=3.05, ns=0.965, r=0, w = -1, wa = 0):
    """ Generate a temperature power spectrum given cosmological parameters"""
    As = 1e-10 * np.exp(logA)
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0, ombh2=ombh2, omch2=omch2, mnu=mnu, omk=omk, tau=tau)
    pars.InitPower.set_params(As=As, ns=ns, r=r)
    if (w != -1 or wa != 0):
        pars.DarkEnergy = DarkEnergyPPF(w=w, wa=wa)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    pars.NonLinear = camb.model.NonLinear_none
    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')
    totCL = powers['total']
    return totCL[2:, 0]

obs = generate_camb_cl()
obs_pars = [67.5, 0.022, 0.122, 3.05, 0.965]

In [9]:
obs_sample = swyft.Sample(x = torch.tensor(obs), z = torch.tensor(obs_pars))
samples = swyft.Samples(x = torch.tensor(cls_short), z = torch.tensor(pars))
dm_full = swyft.SwyftDataModule(samples, batch_size = 64)

/tmp/ipykernel_4575/2246489196.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  samples = swyft.Samples(x = torch.tensor(cls_short), z = torch.tensor(pars))


In [10]:
maxs = np.array([90, 0.05, 0.5, 3.5, 1])
mins = np.array([50, 0.01, 0.01, 1.5, 0.8])
N = 10000
H0_rand = np.random.uniform(mins[0], maxs[0], N)
ombh2_rand = np.random.uniform(mins[1], maxs[1], N)
omch2_rand = np.random.uniform(mins[2], maxs[2], N)
logA_rand = np.random.uniform(mins[3], maxs[3], N)
ns_rand = np.random.uniform(mins[4], maxs[4], N)

z_prior = []
for i in range(N):
  z_prior.append([H0_rand[i], ombh2_rand[i], omch2_rand[i], logA_rand[i], ns_rand[i]])
prior_samples = swyft.Samples(z = torch.tensor(z_prior))


In [11]:
#import tensorflow as tf
#from pytorch_lightning import loggers as pl_loggers
#from pytorch_lightning.callbacks.early_stopping import EarlyStopping
#from pytorch_lightning.callbacks import LearningRateMonitor

#logger = pl_loggers.TensorBoardLogger(save_dir='logs/', name='test', version=None)
#early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta = 0., patience=1, verbose=True, mode='min')
#lr_monitor = LearningRateMonitor(logging_interval='step')

#trainer3 = swyft.SwyftTrainer(accelerator = DEVICE, precision = 64, logger = logger, callbacks=[lr_monitor,early_stopping_callback])
trainer3 = swyft.SwyftTrainer(accelerator = DEVICE, precision = 64)
network3 = Network3()
trainer3.fit(network3, dm_full)
predictions3 = trainer3.infer(network3, obs_sample, prior_samples)
truth = {k: v for k, v in zip(["z[%i]"%i for i in range(5)], obs_sample['z'])}
swyft.plot_posterior(predictions3, ["z[%i]"%i for i in range(5)], truth=truth);

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/lightning_fabric/accelerators/cuda.py:235: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint

  | Name      | Type                   | Params
-----------------------------------------------------
0 | net       | Sequential             | 339 K 
1 | logratios | LogRatioEstimator_1dim | 89.3 K
-----------------------------------------------------
429 K     Trainable params
0         Non